# 复杂关系查询

In [1]:
# 连接neo4j
from py2neo import Graph
graph = Graph("bolt: // localhost:7687", auth=("neo4j", "12345678"))
from py2neo import Node
from py2neo import Relationship
from py2neo import NodeMatcher

## 查询两个明星间的最短路径

In [7]:
# 查询最短路径
query = "MATCH p = shortestpath((s1:明星 {name:'周迅'})-[*..6]-(s2:明星 {name:'王菲'})) RETURN p"
# 执行 cypher 语句
graph.run(query)

p
(周迅)<-[:涉及 {}]-(综艺《幻乐之城》)-[:涉及 {}]->(王菲)


## 查找与周迅有共同好友的明星

In [14]:
query = "MATCH (s1:明星 {name: '周迅'})-[:好友]-(f:明星)-[:好友]-(s2:明星) WHERE s2 <> s1 RETURN DISTINCT s2.name as 明星"
graph.run(query)

明星
王菲


## 查找发生在香港的事件以及涉及到的明星

In [25]:
query = "MATCH (p:地点 {name: '香港'})<-[r:发生在]-(e:事件)-[r2:涉及]-(s:明星) RETURN DISTINCT e.name as 事件,s.name as 明星"
result = graph.run(query)
for i in result :
    print(i)

'王菲与李亚鹏离婚'	'李亚鹏'
'王菲与李亚鹏离婚'	'王菲'
'谢霆锋张柏芝离婚'	'张柏芝'
'谢霆锋张柏芝离婚'	'谢霆锋'
'艳照门事件'	'谢霆锋'
'艳照门事件'	'陈冠希'
'艳照门事件'	'张柏芝'


## 查找与王菲有关系的事件及涉及到的其他明星：

In [26]:
query = "MATCH (s:明星 {name: '王菲'})-[:涉及]-(e:事件)-[:涉及]-(otherStar:明星) RETURN e.name as 事件,otherStar.name as 涉及到的明星"
result = graph.run(query)
for i in result :
    print(i)

'王菲与李亚鹏离婚'	'李亚鹏'
'春晚节目《相约一九九八》'	'那英'
'综艺《幻乐之城》'	'周迅'
'综艺《幻乐之城》'	'窦靖童'


## 匹配所有与陈冠希和张柏芝有关的节点，同时过滤掉谢霆锋

In [27]:
query = "match p=(n)--(m) where n.name in ['陈冠希','张柏芝'] and not m.name in ['谢霆锋'] return distinct m.name"
result = graph.run(query)
for i in result :
    print(i)

'艳照门事件'
'加拿大'
'张柏芝'
'谢霆锋张柏芝离婚'
'中国'
'香港'
'陈冠希'
'那英'


## 查询周迅的旧爱们

In [28]:
query = "match p=(n)-[r:旧爱]->(m) where n.name ='周迅' return m.name"
result = graph.run(query)
for i in result :
    print(i)

'窦鹏'
'李大齐'
'朴树'
'李亚鹏'


## 查询陈冠希和张亚东之间有无关系

In [31]:
query = "optional match p=(m)-[*..8]->(n) where m.name ='陈冠希' and n.name ='张亚东' return p"
result = graph.run(query)
for i in result :
    print(i)

Path(Node('明星', age=40, name='陈冠希', sex='男', tag='life is a fxxking movie'), 旧爱(Node('明星', age=40, name='陈冠希', sex='男', tag='life is a fxxking movie'), Node('明星', age=42, name='张柏芝', sex='女')), 离异(Node('明星', age=42, name='张柏芝', sex='女'), Node('明星', age=42, name='谢霆锋', sex='男')), 旧友(Node('明星', age=42, name='谢霆锋', sex='男'), Node('明星', age=40, name='陈冠希', sex='男', tag='life is a fxxking movie')), 旧友(Node('明星', age=40, name='陈冠希', sex='男', tag='life is a fxxking movie'), Node('明星', age=42, name='谢霆锋', sex='男')), 绯闻(Node('明星', age=42, name='谢霆锋', sex='男'), Node('明星', age=46, name='周迅', sex='女')), 旧爱(Node('明星', age=46, name='周迅', sex='女'), Node('明星', age=52, name='李亚鹏', sex='男')), 旧爱(Node('明星', age=52, name='李亚鹏', sex='男'), Node('明星', age=52, name='翟颖', sex='女')), 旧爱(Node('明星', age=52, name='翟颖', sex='女'), Node('明星', age=55, name='张亚东', sex='男')))
Path(Node('明星', age=40, name='陈冠希', sex='男', tag='life is a fxxking movie'), 旧爱(Node('明星', age=40, name='陈冠希', sex='男', tag='life is a fxxking mov

Path(Node('明星', age=40, name='陈冠希', sex='男', tag='life is a fxxking movie'), 旧爱(Node('明星', age=40, name='陈冠希', sex='男', tag='life is a fxxking movie'), Node('明星', age=42, name='张柏芝', sex='女')), 离异(Node('明星', age=42, name='张柏芝', sex='女'), Node('明星', age=42, name='谢霆锋', sex='男')), 绯闻(Node('明星', age=42, name='谢霆锋', sex='男'), Node('明星', age=46, name='周迅', sex='女')), 好友(Node('明星', age=46, name='周迅', sex='女'), Node('明星', age=55, name='那英', sex='女')), 好友(Node('明星', age=55, name='那英', sex='女'), Node('明星', age=46, name='周迅', sex='女')), 旧爱(Node('明星', age=46, name='周迅', sex='女'), Node('明星', age=52, name='李亚鹏', sex='男')), 旧爱(Node('明星', age=52, name='李亚鹏', sex='男'), Node('明星', age=52, name='翟颖', sex='女')), 旧爱(Node('明星', age=52, name='翟颖', sex='女'), Node('明星', age=55, name='张亚东', sex='男')))
Path(Node('明星', age=40, name='陈冠希', sex='男', tag='life is a fxxking movie'), 旧爱(Node('明星', age=40, name='陈冠希', sex='男', tag='life is a fxxking movie'), Node('明星', age=42, name='张柏芝', sex='女')), 离异(Node('明星', age=

Path(Node('明星', age=40, name='陈冠希', sex='男', tag='life is a fxxking movie'), 旧爱(Node('明星', age=40, name='陈冠希', sex='男', tag='life is a fxxking movie'), Node('明星', age=42, name='张柏芝', sex='女')), 旧爱(Node('明星', age=42, name='张柏芝', sex='女'), Node('明星', age=40, name='陈冠希', sex='男', tag='life is a fxxking movie')), 旧友(Node('明星', age=40, name='陈冠希', sex='男', tag='life is a fxxking movie'), Node('明星', age=42, name='谢霆锋', sex='男')), 绯闻(Node('明星', age=42, name='谢霆锋', sex='男'), Node('明星', age=46, name='周迅', sex='女')), 旧爱(Node('明星', age=46, name='周迅', sex='女'), Node('明星', age=51, name='窦鹏', sex='男')), 堂兄弟(Node('明星', age=51, name='窦鹏', sex='男'), Node('明星', age=54, name='窦唯', sex='男')), 兄妹(Node('明星', age=54, name='窦唯', sex='男'), Node('明星', age=52, name='窦颖', sex='女')), 离异(Node('明星', age=52, name='窦颖', sex='女'), Node('明星', age=55, name='张亚东', sex='男')))
Path(Node('明星', age=40, name='陈冠希', sex='男', tag='life is a fxxking movie'), 旧爱(Node('明星', age=40, name='陈冠希', sex='男', tag='life is a fxxking movie'

Path(Node('明星', age=40, name='陈冠希', sex='男', tag='life is a fxxking movie'), 旧友(Node('明星', age=40, name='陈冠希', sex='男', tag='life is a fxxking movie'), Node('明星', age=42, name='谢霆锋', sex='男')), 绯闻(Node('明星', age=42, name='谢霆锋', sex='男'), Node('明星', age=46, name='周迅', sex='女')), 旧爱(Node('明星', age=46, name='周迅', sex='女'), Node('明星', age=51, name='窦鹏', sex='男')), 旧爱(Node('明星', age=51, name='窦鹏', sex='男'), Node('明星', age=46, name='周迅', sex='女')), 旧爱(Node('明星', age=46, name='周迅', sex='女'), Node('明星', age=52, name='李亚鹏', sex='男')), 旧爱(Node('明星', age=52, name='李亚鹏', sex='男'), Node('明星', age=52, name='翟颖', sex='女')), 旧爱(Node('明星', age=52, name='翟颖', sex='女'), Node('明星', age=55, name='张亚东', sex='男')))
Path(Node('明星', age=40, name='陈冠希', sex='男', tag='life is a fxxking movie'), 旧友(Node('明星', age=40, name='陈冠希', sex='男', tag='life is a fxxking movie'), Node('明星', age=42, name='谢霆锋', sex='男')), 绯闻(Node('明星', age=42, name='谢霆锋', sex='男'), Node('明星', age=46, name='周迅', sex='女')), 旧爱(Node('明星', age=4

Path(Node('明星', age=40, name='陈冠希', sex='男', tag='life is a fxxking movie'), 旧友(Node('明星', age=40, name='陈冠希', sex='男', tag='life is a fxxking movie'), Node('明星', age=42, name='谢霆锋', sex='男')), 绯闻(Node('明星', age=42, name='谢霆锋', sex='男'), Node('明星', age=46, name='周迅', sex='女')), 好友(Node('明星', age=46, name='周迅', sex='女'), Node('明星', age=55, name='那英', sex='女')), 好友(Node('明星', age=55, name='那英', sex='女'), Node('明星', age=46, name='周迅', sex='女')), 旧爱(Node('明星', age=46, name='周迅', sex='女'), Node('明星', age=51, name='窦鹏', sex='男')), 堂兄弟(Node('明星', age=51, name='窦鹏', sex='男'), Node('明星', age=54, name='窦唯', sex='男')), 兄妹(Node('明星', age=54, name='窦唯', sex='男'), Node('明星', age=52, name='窦颖', sex='女')), 离异(Node('明星', age=52, name='窦颖', sex='女'), Node('明星', age=55, name='张亚东', sex='男')))
Path(Node('明星', age=40, name='陈冠希', sex='男', tag='life is a fxxking movie'), 旧友(Node('明星', age=40, name='陈冠希', sex='男', tag='life is a fxxking movie'), Node('明星', age=42, name='谢霆锋', sex='男')), 绯闻(Node('明星', age=42,

Path(Node('明星', age=40, name='陈冠希', sex='男', tag='life is a fxxking movie'), 旧友(Node('明星', age=40, name='陈冠希', sex='男', tag='life is a fxxking movie'), Node('明星', age=42, name='谢霆锋', sex='男')), 旧爱(Node('明星', age=42, name='谢霆锋', sex='男'), Node('明星', age=53, name='王菲', sex='女')), 离异(Node('明星', age=53, name='王菲', sex='女'), Node('明星', age=54, name='窦唯', sex='男')), 兄妹(Node('明星', age=54, name='窦唯', sex='男'), Node('明星', age=52, name='窦颖', sex='女')), 离异(Node('明星', age=52, name='窦颖', sex='女'), Node('明星', age=55, name='张亚东', sex='男')))
Path(Node('明星', age=40, name='陈冠希', sex='男', tag='life is a fxxking movie'), 旧友(Node('明星', age=40, name='陈冠希', sex='男', tag='life is a fxxking movie'), Node('明星', age=42, name='谢霆锋', sex='男')), 旧爱(Node('明星', age=42, name='谢霆锋', sex='男'), Node('明星', age=53, name='王菲', sex='女')), 离异(Node('明星', age=53, name='王菲', sex='女'), Node('明星', age=54, name='窦唯', sex='男')), 兄妹(Node('明星', age=54, name='窦唯', sex='男'), Node('明星', age=52, name='窦颖', sex='女')), 离异(Node('明星', age=52,